In [1]:
import numpy as np
import cv2 as cv
import cv2
import glob
import depthai  # depthai - access the camera and its data packets
import blobconverter  # blobconverter - compile and download MyriadX neural network blobs
import pickle


In [3]:
pipeline = depthai.Pipeline()
cam_rgb = pipeline.create(depthai.node.ColorCamera)
cam_rgb.setPreviewSize(1280,720)
cam_rgb.setInterleaved(False)

#Creating a stream of the rgb camera and naming it "rgb"
xout_rgb = pipeline.create(depthai.node.XLinkOut)
xout_rgb.setStreamName("rgb")
cam_rgb.preview.link(xout_rgb.input)


In [4]:
frame = None
coords = []
def click_event(event, x, y, flags, params):
    # checking for left mouse clicks
    if event == cv2.EVENT_LBUTTONDOWN:
        coords.append((x, y))
        if len(coords) == 3:
            coords.pop(0)

In [ ]:
#import variables from the pickle file
with open('calibration.pkl', 'rb') as f:
    calibration = pickle.load(f)
ret = calibration['ret']
mtx = calibration['mtx']
dist = calibration['dist']
rvecs = calibration['rvecs']
tvecs = calibration['tvecs']

In [6]:
cv2.namedWindow("image")
cv2.setMouseCallback('image', click_event)
_cm = 35
_scale = 10
z_dist = _cm*_scale#cm
with depthai.Device(pipeline) as device:
    try:
        calibData = device.readCalibration2()
        lensPosition = calibData.getLensPosition(depthai.CameraBoardSocket.RGB)
        intrinsics = calibData.getCameraIntrinsics(depthai.CameraBoardSocket.RGB)
        print("intrinsics: ", mtx)
        np_intrinsics = np.array(mtx)
        np_intrinsics=np.linalg.inv(np_intrinsics)

        # print("intrinsics: ", intrinsics)
        if lensPosition:
            cam_rgb.initialControl.setManualFocus(lensPosition)
    except:
        raise
    # From this point, the Device will be in "running" mode and will start sending data via XLink

    # To consume the device results, we get two output queues from the device, with stream names we assigned earlier
    q_rgb = device.getOutputQueue("rgb")

    # Here, some of the default values are defined. Frame will be an image from "rgb" stream, detections will contain nn results
    frame = None
    detections = []

    # Since the detections returned by nn have values from <0..1> range, they need to be multiplied by frame width/height to
    # receive the actual position of the bounding box on the image
    def frameNorm(frame, bbox):
        normVals = np.full(len(bbox), frame.shape[0])
        normVals[::2] = frame.shape[1]
        return (np.clip(np.array(bbox), 0, 1) * normVals).astype(int)

    img_counter = 0
    # Main host-side application loop
    while True:
        # we try to fetch the data from nn/rgb queues. tryGet will return either the data packet or None if there isn't any
        in_rgb = q_rgb.tryGet()

        if in_rgb is not None:
            # If the packet from RGB camera is present, we're retrieving the frame in OpenCV format using getCvFrame
            frame = in_rgb.getCvFrame()
            # print(frame.shape)


        if frame is not None:
            for c in coords:
                cv2.circle(frame, c, 5, (255, 0, 0), -1)
                if(len(coords)==2):
                    cv2.line(frame, coords[0], coords[1], (255, 0, 0), 5)
                    dist = np.linalg.norm(np.array(coords[0])-np.array(coords[1]))
                    
                    
                    pix_coord_1 = np.array([coords[0][0], coords[0][1], z_dist])
                    pix_coord_2 = np.array([coords[1][0], coords[1][1], z_dist])

                    real_coord_1 = pix_coord_1 @ np_intrinsics
                    real_coord_2 = pix_coord_2 @ np_intrinsics 
                    real_dist = np.linalg.norm(real_coord_1-real_coord_2)/_scale
                    #cv2.putText(frame, "cord 1: " + str(real_coord_1), (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    cv2.putText(frame, "real_dist: " + str(real_dist) + " cm", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

            # After all the drawing is finished, we show the frame on the screen
            cv2.imshow("image", frame)

        # at any time, you can press "q" and exit the main loop, therefore exiting the program itself
        k = cv.waitKey(1)
        if k%256 == 27:
            # ESC pressed
            print("Escape hit, closing...")
            break

cv2.destroyAllWindows()

intrinsics:  [[897.65110375   0.         559.40540049]
 [  0.         885.60904869 374.39506256]
 [  0.           0.           1.        ]]


RuntimeError: Communication exception - possible device error/misconfiguration. Original message 'Couldn't read data from stream: 'rgb' (X_LINK_ERROR)'

: 